<a href="https://colab.research.google.com/github/mjavadzadeh11/Crawler/blob/master/GoogleNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install GoogleNews
!pip install newspaper3k
!pip install vaderSentiment;

In [ ]:
!pip install googleanalytics


In [ ]:
import pandas as pd
from GoogleNews import GoogleNews
from newspaper import Article
import requests
import nltk
nltk.download('punkt')
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
import googleanalytics

In [ ]:
googlenews = GoogleNews(period='30d')
#googlenews = GoogleNews(start='11/04/2021',end='12/04/2021')
googlenews.search('Bitcoin')
googlenews.get_page(30)
Result=googlenews.results()

In [ ]:
def get_news(keyword , period, NumOfPages):
#  ''' keyword is something that will be searched in google , period is the period of search
#  and NumOfPages is how many pages def should search'''
  googlenews = GoogleNews(period='30d')
  googlenews.search('Bitcoin')
  for i in range(2,NumOfPages):
    googlenews.getpage(i)
    result=googlenews.result()
    df=pd.DataFrame(result)
  return df

df=get_news('Bitcoin', '30d',20)

In [ ]:
df.drop(columns=['datetime'], inplace=True)

In [ ]:
df.drop(columns=['img'], inplace=True)

In [ ]:
df



In [ ]:
#filtering by media
filt = df['media']== 'Al Jazeera'
df[filt]

In [ ]:
#grouping by media
media_gp= df.groupby(['media'])
media_gp ['link'].value_counts() 

In [ ]:

for index in range(df.shape[0]):
  try:
    article=Article(df.loc[index,'link'])
    article.download()
    article.html
    article.parse()
    article.nlp()
    
    
    df.loc[index,'Context'] = article.text
    df.loc[index, 'Summary'] = article.summary
    df.loc[index, 'Keywords'] = ' '.join(article.keywords)
  except:
    df.loc[index,'Context'] = 'Not able to recieve this news'
    df.loc[index, 'Summary'] = 'Could not summarize the context'
    df.loc[index, 'Keywords'] = "No keywords founded"
    

#df['Context'] = news_context




In [ ]:
df.head()

In [ ]:
#sentiment analysis
#Analyzer = SentimentIntensityAnalyzer()
#negative=[]
#netrual=[]
#positive=[]
#for i in range(df.shape[0]):
#  title = df.iloc[i,0]
#  desc	 = df.iloc[i,4]
#  summ = df.loc[i,'Summary']
#  title_analyzer = Analyzer.polarity_scores(title)
#  caption_analyzer = Analyzer.polarity_scores(desc)
#  summary_analyzer = Analyzer.polarity_scores(summ)
#  negative.append(((title_analyzer['neg']) + (caption_analyzer['neg']) + (summary_analyzer['neg']) )/3)
#  positive.append(((title_analyzer['pos']) + (caption_analyzer['pos']) + (summary_analyzer['pos']))/3)
#  netrual.append(((title_analyzer['neu']) + (caption_analyzer['neu'])+ (summary_analyzer['neu']))/3)

#df['Negative']= negative
#df['Positive']= positive
#df['Netrual'] = netrual

In [ ]:
from textblob import TextBlob

In [ ]:
polarity_list=[]
for text in df.loc[:,'Context']:
  blob = TextBlob(text)
  polarity_list.append(blob.sentiment.polarity)



In [ ]:
df["Polarity"]=polarity_list

In [ ]:
df.to_csv('news1.csv')

In [ ]:
df.nsmallest(5,['Polarity'])